In [1]:
import re
import numpy as np
from enum import Enum
from multiprocessing import Pool


In [2]:
input = '''Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian.
'''
input = input.replace('\n  ', ' ').replace('\n\n', '\n')

# with open('input') as f:
#     input = f.read()


In [3]:
MAX_TIME = 24

RESOURCE_I = Enum('Resources', ['ORE', 'CLAY', 'OBSIDIAN', 'GEODE'], start=0)


In [4]:
class Blueprint:
    id = 0
    costs_ore = [0] * 4
    costs_clay = [0] * 4
    costs_obsidian = [0] * 4
    costs_geode = [0] * 4


In [5]:
blueprints = list()
for line in input.split('\n'):
    if len(line) < 1:
        continue

    title, plan = line.split(': ')

    m = re.match('Blueprint (\\d+)', title)
    bp = Blueprint()
    bp.id = int(m.group(1))

    for robotCosts in plan.split('. '):
        m = re.match('Each (\\w+) robot costs ([\\w ]+)', robotCosts)
        robotName = m.group(1)
        costStr = m.group(2).split(' and ')

        costs = [0] * 4
        for e in costStr:
            c = re.match('(\\d+) (\\w+)', e)
            costs[RESOURCE_I[str.upper(c.group(2))].value] = int(c.group(1))
        assert costs != [0] * 4

        match robotName:
            case 'ore':
                bp.costs_ore = costs
            case 'clay':
                bp.costs_clay = costs
            case 'obsidian':
                bp.costs_obsidian = costs
            case 'geode':
                bp.costs_geode = costs
            case _:
                assert False
    blueprints.append(bp)


In [6]:
def dp(time, resources, robots, blueprint):
    if time > MAX_TIME:
        return resources[RESOURCE_I['GEODE'].value]

    bsf = -1
    nextResources = np.add(resources, robots)
    lastResources = np.subtract(resources, robots)

    # build geode-collecting robot
    if np.all(np.greater_equal(resources, blueprint.costs_geode)):
        if not np.all(np.greater_equal(lastResources, blueprint.costs_geode)):
            bsf = max(bsf, dp(time + 1, np.subtract(nextResources, blueprint.costs_geode), np.add(robots, [0, 0, 0, 1]), blueprint))

    # build obsidian-collecting robot
    if np.all(np.greater_equal(resources, blueprint.costs_obsidian)):
        if not np.all(np.greater_equal(lastResources, blueprint.costs_obsidian)):
            if robots[RESOURCE_I['OBSIDIAN'].value] < max([
                blueprint.costs_ore[RESOURCE_I['OBSIDIAN'].value],
                blueprint.costs_clay[RESOURCE_I['OBSIDIAN'].value],
                blueprint.costs_obsidian[RESOURCE_I['OBSIDIAN'].value],
                blueprint.costs_geode[RESOURCE_I['OBSIDIAN'].value]
            ]):
                bsf = max(bsf, dp(time + 1, np.subtract(nextResources, blueprint.costs_obsidian), np.add(robots, [0, 0, 1, 0]), blueprint))

    # build clay-collecting robot
    if np.all(np.greater_equal(resources, blueprint.costs_clay)):
        if not np.all(np.greater_equal(lastResources, blueprint.costs_clay)):
            if robots[RESOURCE_I['CLAY'].value] < max([
                blueprint.costs_ore[RESOURCE_I['CLAY'].value],
                blueprint.costs_clay[RESOURCE_I['CLAY'].value],
                blueprint.costs_obsidian[RESOURCE_I['CLAY'].value],
                blueprint.costs_geode[RESOURCE_I['CLAY'].value]
            ]):
                bsf = max(bsf, dp(time + 1, np.subtract(nextResources, blueprint.costs_clay), np.add(robots, [0, 1, 0, 0]), blueprint))

    # build ore-collecting robot
    if np.all(np.greater_equal(resources, blueprint.costs_ore)):
        if not np.all(np.greater_equal(lastResources, blueprint.costs_ore)):
            if robots[RESOURCE_I['ORE'].value] < max([
                blueprint.costs_ore[RESOURCE_I['ORE'].value],
                blueprint.costs_clay[RESOURCE_I['ORE'].value],
                blueprint.costs_obsidian[RESOURCE_I['ORE'].value],
                blueprint.costs_geode[RESOURCE_I['ORE'].value]
            ]):
                bsf = max(bsf, dp(time + 1, np.subtract(nextResources, blueprint.costs_ore), np.add(robots, [1, 0, 0, 0]), blueprint))

    # do not build any robot
    if np.any(np.less(resources, np.maximum.reduce([blueprint.costs_ore, blueprint.costs_clay, blueprint.costs_obsidian, blueprint.costs_geode]))):
        remaining_time = MAX_TIME - time
        if nextResources[RESOURCE_I['GEODE'].value] + ((remaining_time - 1) * remaining_time / 2) > bsf:
            bsf = max(bsf, dp(time + 1, nextResources, robots, blueprint))

    return bsf


In [7]:
def ql(blueprint):
    return blueprint.id * dp(1, [0, 0, 0, 0], [1, 0, 0, 0], blueprint)


In [8]:
qualityLevel = list()
with Pool() as p:
    qualityLevel = list(p.map(ql, blueprints))


In [9]:
print(sum(qualityLevel))


33
